In [ ]:
# pip install lazypredict
# pip install imbalanced-learn
# pip install optuna

In [ ]:
from lazypredict.Supervised import LazyClassifier  # use LazyRegressor for regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

import pandas as pd


In [ ]:
import optuna
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestCentroid
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
sample_df=pd.read_csv("D:/DA-EBPL/Sample_submission.csv")

In [ ]:
Train_df=pd.read_csv("D:/DA-EBPL/Train.csv")

In [ ]:
Test_df=pd.read_csv("D:/DA-EBPL/Test.csv")

In [ ]:
Train_df.head()

In [ ]:
Test_df.head()

In [ ]:
sample_df.head()

In [ ]:
Train_df.shape

In [ ]:
## EDA

In [ ]:
Train_df["Class"].value_counts()

In [ ]:
Bool_cols=[ i for i in Train_df.columns.to_list() if Train_df[i].nunique()==2]

In [ ]:
len(Bool_cols)

In [ ]:
Other_cols=set(Train_df.columns.to_list())-set(Bool_cols)

In [ ]:
len(Other_cols)

In [ ]:
Train_df[list(Bool_cols)].sum()

In [ ]:
Train_df["feature_4"].nunique()

In [ ]:
No_variance_column=[i for i in Other_cols if Train_df[i].nunique() ==1]

In [ ]:
Train_df[list(No_variance_column)].sum()

In [ ]:
Train_df[list(Other_cols)].describe()

In [ ]:
Valid_other_cols=set(Other_cols)-set(No_variance_column)

In [ ]:
len(Valid_other_cols)

In [ ]:
Train_df[list(Valid_other_cols)].corr()

In [ ]:
bool_corr=Train_df[list(Bool_cols)].corr()

In [ ]:
bool_corr

In [ ]:
import pandas as pd

def get_high_correlations(df, threshold=0.5):
    corr = df.corr()

    # Unstack and reset index
    corr_pairs = corr.unstack().reset_index()
    corr_pairs.columns = ['Feature1', 'Feature2', 'Correlation']

    # Remove self-correlations
    corr_pairs = corr_pairs[corr_pairs['Feature1'] != corr_pairs['Feature2']]

    # Remove duplicate pairs (A-B and B-A)
    corr_pairs['Sorted'] = corr_pairs.apply(lambda x: tuple(sorted([x['Feature1'], x['Feature2']])), axis=1)
    corr_pairs = corr_pairs.drop_duplicates('Sorted').drop(columns='Sorted')

    # Filter by threshold
    filtered = corr_pairs[(corr_pairs['Correlation'] > threshold) | (corr_pairs['Correlation'] < threshold*-1)]

    return filtered


In [ ]:
bool_corr=get_high_correlations(df=Train_df[list(Bool_cols)], threshold=0.95)

In [ ]:
bool_corr

In [ ]:
dict1={}
related_feature=[]
for i in list(bool_corr.query("Correlation== 1").Feature1.unique()):
  if i not in related_feature:
    dict1[i]=list(bool_corr[bool_corr.Feature1==i].Feature2.unique())
    related_feature.extend(list(bool_corr[bool_corr.Feature1==i].Feature2.unique()))


In [ ]:
len(dict1.keys())

In [ ]:
len(dict1.keys())

In [ ]:
len(bool_corr.query("Correlation== 1").Feature1.unique())-len(dict1.keys())

In [ ]:
Co_related_columns=list(set(bool_corr.query("Correlation== 1").Feature1.unique())-set(dict1.keys()))

In [ ]:
len(No_variance_column)

In [ ]:
len(Co_related_columns+list(No_variance_column))

In [ ]:
No_variance_column,

In [ ]:
Train_df.shape[1]-553-1-36-3 # valid Bool Columns

In [ ]:
Train_df[["Class","feature_1"]].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Example: df['temperature'] is numeric, df['anomaly'] is 0 or 1
sns.histplot(data=Train_df, x='feature_1', hue='Class', kde=True, stat='density', common_norm=False)

plt.title("Class Distribution by Anomaly Status")
plt.xlabel("feature_1")
plt.ylabel("Density")
plt.legend(title='Class', labels=['Normal (0)', 'Anomaly (1)'])
plt.show()

In [ ]:
# Example: 'temperature' is numeric, 'anomaly' is binary (0 or 1)
sns.boxplot(data=Train_df, x='Class', y='feature_1', palette='Set2')

plt.title("Class Distribution by Anomaly Status")
plt.ylabel("feature_1")
plt.xlabel("Class (0 = Normal, 1 = Anomaly)")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:
# Feature Engineering (Remove or Modified Unwanted columns)

In [ ]:
Train_dataset=Train_df.drop(columns=Co_related_columns+list(No_variance_column))

In [ ]:
X=Train_dataset.drop(columns="Class")
y=Train_dataset["Class"]

In [ ]:
# Split manually, if not already split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [ ]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, y_train, X_test, y_test)
display(models)

In [ ]:
# prediction Model Fine tuning

In [ ]:
def lazy_classifier_metrics_from_split(
    X_train, X_test, y_train, y_test, top_n=10, calculate_roc_auc=True
):
    """
    Evaluate top N LazyClassifier models with train/test metrics for binary classification.
    Checks only for NaNs in predicted probabilities.
    """

    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)

    top_models = models.head(top_n)

    metrics = []

    for model_name in top_models.index:
        model = clf.models[model_name]
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        if calculate_roc_auc:
            try:
                y_train_proba = model.predict_proba(X_train)[:, 1]
                y_test_proba = model.predict_proba(X_test)[:, 1]

                if pd.isna(y_train_proba).any() or pd.isna(y_test_proba).any():
                    print(f"Warning: NaNs detected in predicted probabilities for model {model_name}")

                train_roc = roc_auc_score(y_train, y_train_proba)
                test_roc = roc_auc_score(y_test, y_test_proba)

            except (AttributeError, NotImplementedError, ValueError) as e:
                print(f"Warning: Could not compute ROC AUC for model {model_name}: {e}")
                train_roc = None
                test_roc = None
        else:
            train_roc = None
            test_roc = None

        metrics.append({
            "Model": model_name,
            "Test Accuracy": accuracy_score(y_test, y_test_pred),
            "Train Accuracy": accuracy_score(y_train, y_train_pred),
            "Test F1 Score": f1_score(y_test, y_test_pred, zero_division=0),
            "Train F1 Score": f1_score(y_train, y_train_pred, zero_division=0),
            "Test Precision": precision_score(y_test, y_test_pred, zero_division=0),
            "Train Precision": precision_score(y_train, y_train_pred, zero_division=0),
            "Test Recall": recall_score(y_test, y_test_pred, zero_division=0),
            "Train Recall": recall_score(y_train, y_train_pred, zero_division=0),
            "Test ROC AUC": test_roc,
            "Train ROC AUC": train_roc
        })

    results_df = pd.DataFrame(metrics)[[
        "Model",
        "Test Accuracy", "Train Accuracy",
        "Test F1 Score", "Train F1 Score",
        "Test Precision", "Train Precision",
        "Test Recall", "Train Recall",
        "Test ROC AUC", "Train ROC AUC"
    ]]

    return results_df.round(4)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
def lazy_classifier_metrics_from_split_standard_scaler(
    X_train, X_test, y_train, y_test, top_n=10, calculate_roc_auc=True
):
    """
    Evaluate top N LazyClassifier models with train/test metrics for binary classification.
    Checks only for NaNs in predicted probabilities.
    """
    scaler = StandardScaler()

    # Fit on train data only
    X_train_scaled = scaler.fit_transform(X_train)

    # Transform test data
    X_test_scaled = scaler.transform(X_test)

    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models, predictions = clf.fit(X_train_scaled, X_test_scaled, y_train, y_test)

    top_models = models.head(top_n)
    display(top_models)

    metrics = []

    for model_name in top_models.index:
        model = clf.models[model_name]
        model.fit(X_train_scaled, y_train)

        y_train_pred = model.predict(X_train_scaled)
        y_test_pred = model.predict(X_test_scaled)

        if calculate_roc_auc:
            try:
                y_train_proba = model.predict_proba(X_train_scaled)[:, 1]
                y_test_proba = model.predict_proba(X_test_scaled)[:, 1]

                if pd.isna(y_train_proba).any() or pd.isna(y_test_proba).any():
                    print(f"Warning: NaNs detected in predicted probabilities for model {model_name}")

                train_roc = roc_auc_score(y_train, y_train_proba)
                test_roc = roc_auc_score(y_test, y_test_proba)

            except (AttributeError, NotImplementedError, ValueError) as e:
                print(f"Warning: Could not compute ROC AUC for model {model_name}: {e}")
                train_roc = None
                test_roc = None
        else:
            train_roc = None
            test_roc = None

        metrics.append({
            "Model": model_name,
            "Test Accuracy": accuracy_score(y_test, y_test_pred),
            "Train Accuracy": accuracy_score(y_train, y_train_pred),
            "Test F1 Score": f1_score(y_test, y_test_pred, zero_division=0),
            "Train F1 Score": f1_score(y_train, y_train_pred, zero_division=0),
            "Test Precision": precision_score(y_test, y_test_pred, zero_division=0),
            "Train Precision": precision_score(y_train, y_train_pred, zero_division=0),
            "Test Recall": recall_score(y_test, y_test_pred, zero_division=0),
            "Train Recall": recall_score(y_train, y_train_pred, zero_division=0),
            "Test ROC AUC": test_roc,
            "Train ROC AUC": train_roc
        })

    results_df = pd.DataFrame(metrics)[[
        "Model",
        "Test Accuracy", "Train Accuracy",
        "Test F1 Score", "Train F1 Score",
        "Test Precision", "Train Precision",
        "Test Recall", "Train Recall",
        "Test ROC AUC", "Train ROC AUC"
    ]]

    return results_df.round(4)

In [ ]:
X_train.isna().sum().sum(),X_test.isna().sum().sum(),y_train.isna().sum().sum(),y_test.isna().sum().sum()

In [ ]:
from sklearn.preprocessing import StandardScaler




In [ ]:
# Get metrics
metrics_df_normal = lazy_classifier_metrics_from_split(X_train, X_test, y_train, y_test)
display(metrics_df_normal)



In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


In [ ]:
# Oversampling with SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Undersampling with RandomUnderSampler
under = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = under.fit_resample(X_train, y_train)


In [ ]:

# Run LazyPredict on SMOTE oversampled data
print("=== LazyClassifier with SMOTE (Oversampling) ===")
# Get metrics
metrics_df_smote = lazy_classifier_metrics_from_split(X_train_smote, X_test, y_train_smote, y_test)
display(metrics_df_smote)



In [ ]:

# Run LazyPredict on Undersampled data
print("\n=== LazyClassifier with RandomUnderSampler ===")
metrics_df_under = lazy_classifier_metrics_from_split(X_train_under, X_test, y_train_under, y_test)
display(metrics_df_under)



In [ ]:
# Get metrics
metrics_df_normal_scaled = lazy_classifier_metrics_from_split_standard_scaler(X_train, X_test, y_train, y_test)
display(metrics_df_normal_scaled)



In [ ]:

# Run LazyPredict on SMOTE oversampled data
print("=== LazyClassifier with SMOTE Scaled (Oversampling) ===")
# Get metrics
metrics_df_smote_scaled = lazy_classifier_metrics_from_split_standard_scaler(X_train_smote, X_test, y_train_smote, y_test)
display(metrics_df_smote_scaled)



In [ ]:

# Run LazyPredict on Undersampled data
print("\n=== LazyClassifier with Scaled RandomUnderSampler ===")
metrics_df_under_scaled = lazy_classifier_metrics_from_split_standard_scaler(X_train_under, X_test, y_train_under, y_test)
display(metrics_df_under_scaled)



In [ ]:
def tune_adaboost(X, y, cv=5):
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 300),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
        }
        model = AdaBoostClassifier(**params)
        return cross_val_score(model, X, y, cv=cv, scoring='accuracy').mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20)
    return AdaBoostClassifier(**study.best_params)

def tune_bernoulli_nb(X, y, cv=5):
    def objective(trial):
        params = {
            "alpha": trial.suggest_float("alpha", 0.0, 1.0),
            "binarize": trial.suggest_float("binarize", 0.0, 1.0)
        }
        model = BernoulliNB(**params)
        return cross_val_score(model, X, y, cv=cv, scoring='accuracy').mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20)
    return BernoulliNB(**study.best_params)

def tune_xgb(X, y, cv=5):
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 300),
            "max_depth": trial.suggest_int("max_depth", 2, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0)
        }
        model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", **params)
        return cross_val_score(model, X, y, cv=cv, scoring='accuracy').mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20)
    return XGBClassifier(use_label_encoder=False, eval_metric="logloss", **study.best_params)

def tune_nearest_centroid(X, y, cv=5):
    def objective(trial):
        params = {
            "metric": trial.suggest_categorical("metric", ["euclidean", "manhattan"]),
        }
        model = NearestCentroid(**params)
        return cross_val_score(model, X, y, cv=cv, scoring='accuracy').mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=10)
    return NearestCentroid(**study.best_params)


def tune_random_forest(X, y, cv=5, n_trials=20):
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 2, 30),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        }

        model = RandomForestClassifier(**params, random_state=42)
        score = cross_val_score(model, X, y, cv=cv, scoring="accuracy").mean()
        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)
    return RandomForestClassifier(**study.best_params, random_state=42)


In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name="Model"):
    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    print(f"\n--- {model_name} TRAIN ---")
    print(confusion_matrix(y_train, y_pred_train))
    print(classification_report(y_train, y_pred_train))

    print(f"\n--- {model_name} TEST ---")
    print(confusion_matrix(y_test, y_pred_test))
    print(classification_report(y_test, y_pred_test))


In [ ]:
# AdaBoost
ada_model = tune_adaboost(X_train, y_train)
evaluate_model(ada_model, X_train, y_train, X_test, y_test, "AdaBoost")

In [ ]:
# AdaBoost
ada_model = tune_adaboost(X_train_smote, y_train_smote)
evaluate_model(ada_model, X_train_smote, y_train_smote, X_test, y_test, "AdaBoost")

In [ ]:
# AdaBoost
ada_model = tune_adaboost(X_train_under, y_train_under)
evaluate_model(ada_model, X_train_under, y_train_under, X_test, y_test, "AdaBoost")

In [ ]:


# BernoulliNB
bnb_model = tune_bernoulli_nb(X_train, y_train)
evaluate_model(bnb_model, X_train, y_train, X_test, y_test, "BernoulliNB")


In [ ]:


# BernoulliNB
bnb_model = tune_bernoulli_nb(X_train_smote, y_train_smote)
evaluate_model(bnb_model, X_train_smote, y_train_smote, X_test, y_test, "BernoulliNB")


In [ ]:


# BernoulliNB
bnb_model = tune_bernoulli_nb(X_train_under, y_train_under)
evaluate_model(bnb_model, X_train_under, y_train_under, X_test, y_test, "BernoulliNB")


In [ ]:


# XGBoost
xgb_model = tune_xgb(X_train, y_train)
evaluate_model(xgb_model, X_train, y_train, X_test, y_test, "XGBoost")

In [ ]:


# XGBoost
xgb_model = tune_xgb(X_train_smote, y_train_smote)
evaluate_model(xgb_model, X_train_smote, y_train_smote, X_test, y_test, "XGBoost")

In [ ]:


# XGBoost
xgb_model = tune_xgb(X_train_under, y_train_under)
evaluate_model(xgb_model, X_train_under, y_train_under, X_test, y_test, "XGBoost")

In [ ]:


# Nearest Centroid
nc_model = tune_nearest_centroid(X_train, y_train)
evaluate_model(nc_model, X_train, y_train, X_test, y_test, "NearestCentroid")


In [ ]:


# Nearest Centroid
nc_model = tune_nearest_centroid(X_train_smote, y_train_smote)
evaluate_model(nc_model, X_train_smote, y_train_smote, X_test, y_test, "NearestCentroid")


In [ ]:


# Nearest Centroid
nc_model = tune_nearest_centroid(X_train_under, y_train_under)
evaluate_model(nc_model, X_train_under, y_train_under, X_test, y_test, "NearestCentroid")


In [ ]:
rf_model = tune_random_forest(X_train, y_train)
evaluate_model(rf_model, X_train, y_train, X_test, y_test, "Random Forest")


In [ ]:
rf_model = tune_random_forest(X_train_smote, y_train_smote)
evaluate_model(rf_model, X_train_smote, y_train_smote, X_test, y_test, "Random Forest")


In [ ]:
rf_model = tune_random_forest(X_train_under, y_train_under)
evaluate_model(rf_model, X_train_under, y_train_under, X_test, y_test, "Random Forest")


In [ ]:


# Nearest Centroid
nc_model = tune_nearest_centroid(X_train_smote, y_train_smote)
evaluate_model(nc_model, X_train_smote, y_train_smote, X_test, y_test, "NearestCentroid")


In [ ]:
# Get the centroids
centroids = nc_model.centroids_

# Calculate mean absolute difference between centroids across classes for each feature
importance = np.mean(np.abs(np.diff(centroids, axis=0)), axis=0)

# Display with feature names
import pandas as pd
feature_importance = pd.Series(importance, index=X_train.columns).sort_values(ascending=False)


In [ ]:
# Top 10 features
print(feature_importance[:10])

In [ ]:
Y_test=nc_model.predict(Test_df[list(X_train_smote.columns.to_list())])

In [ ]:
sample_df.value_counts()

In [ ]:
sample_df["Class"]=Y_test

In [ ]:
sample_df.value_counts()

In [ ]:
sample_df.to_csv("D:/DA-EBPL/submission.csv",index=False)